In [1]:
from ib_async import *
util.startLoop()  # uncomment this line when in a notebook
import numpy as np

ib = IB()
id_connection = float(input('Enter the connection id: '))
ib.connect('127.0.0.1', 4002, clientId=id_connection)
contract = Forex('EURGBP')
bars = ib.reqHistoricalData(
        contract,
        endDateTime='',
        durationStr='900 S',
        barSizeSetting='5 secs',
        whatToShow='MIDPOINT',
        useRTH=True,
        formatDate=1,
        keepUpToDate=True)


lookback_window = int(input('Enter the lookback window: '))
number_sdev = int(input('Enter the number of standard deviations: '))

close_prices = [] #crea una lista


def onBarUpdate(bars, hasNewBar):

    position = 0
    close_prices.append(float(bars[-1].close)) #append il prezzo di chiusura alla lista
    ohlc = [] #crea una lista
    ohlc.append(str(bars[-1].date)) #append la data
    ohlc.append(str(bars[-1].close))

    #calculate the bollinger bands:
    if len(close_prices) >= lookback_window:
        std = np.std(close_prices[-lookback_window:]) #calcola la deviazione standard dei 20 prezzi precedenti
        sma = np.mean(close_prices[-lookback_window:])
        upper_band = sma + number_sdev * std #calcola la banda superiore
        lower_band = sma - number_sdev * std #calcola la banda inferiore
        ohlc.append(str(upper_band))
        ohlc.append(str(lower_band))
        ohlc.append(str(sma))
    else:
        ohlc.append('N/A')  # Not enough data for Bollinger Bands
    
    #--------------------------------------------------------------------
    #create strategy's logic:
    if len(close_prices) >= lookback_window:
        if position == 0:
            # ib is not in a position
            if float(bars[-1].close) > float(upper_band):
                position = -1
                ib.placeOrder(contract, MarketOrder('SELL', 100000))
            elif float(bars[-1].close) < float(lower_band):
                position = 1
                ib.placeOrder(contract, MarketOrder('BUY', 100000))
        
        else:
            # ib is in a position
            if position.amount > 0 and float(bars[-1].close) > float(sma):
                position = 0
                ib.placeOrder(contract, MarketOrder('SELL', 100000))
            elif position.amount < 0 and float(bars[-1].close) < float(sma):
                position = 0
                ib.placeOrder(contract, MarketOrder('BUY', 100000))
    #--------------------------------------------------------------------



    print(",".join(ohlc), 'current position is:', position) #printa la lista separata da virgole
    
    



bars.updateEvent += onBarUpdate
ib.run()

#ib.disconnect()

2024-09-03 11:11:15-04:00,0.8429,N/A current position is: 0
2024-09-03 11:11:20-04:00,0.8429,N/A current position is: 0
2024-09-03 11:11:25-04:00,0.8429,N/A current position is: 0
2024-09-03 11:11:30-04:00,0.8429,N/A current position is: 0
2024-09-03 11:11:35-04:00,0.8429,N/A current position is: 0
2024-09-03 11:11:40-04:00,0.8429,N/A current position is: 0
2024-09-03 11:11:45-04:00,0.8429,N/A current position is: 0
2024-09-03 11:11:45-04:00,0.84285,N/A current position is: 0
2024-09-03 11:11:50-04:00,0.84285,N/A current position is: 0
2024-09-03 11:11:50-04:00,0.8428,N/A current position is: 0
2024-09-03 11:11:55-04:00,0.8428,N/A current position is: 0
2024-09-03 11:11:55-04:00,0.8428,N/A current position is: 0
2024-09-03 11:12:00-04:00,0.8428,N/A current position is: 0
2024-09-03 11:12:00-04:00,0.84285,N/A current position is: 0
2024-09-03 11:12:05-04:00,0.84285,N/A current position is: 0
2024-09-03 11:12:05-04:00,0.8428,N/A current position is: 0
2024-09-03 11:12:10-04:00,0.8428,N/A

In [ ]:
closing_prices = [bar.close for bar in bars]
print(closing_prices)